# Demonstration of imputation of outcomes and covariates, followed by cluster-level analysis
Here we extend the previous approaches for cluster-level analyses of RCT data to the situation where we have imputed variables. For the demo we use both outcomes and covariates with missing data, and impute those values using multiple imputation by chained equations (MICE).

In [ ]:
set more off
clear all
capture adopath + "../ado_files/"
use "../../source_data_for_git_notebooks/data/baby_level_cluster_rct_data.dta"
frame rename default main_data
capture frame copy main_data cluster_level, replace

### Note: 
For now, we are replacing weight to missing if it is beyond the time frame (i.e. if `weight_time_met!=1`. Then, we will impute all missing weights. Note that this ignores the information contained in those weights that were measured after 72 hours, and one might consider a different approach (see <a href="doi: 10.1136/bmjopen-2021-060105">Hazel, Zeger, Mullany et al, 2022 BMJ Open. 2022 Jul 12;12(7):e060105.</a>)

In [ ]:
replace weight = . if weight_time_met!=1

Use `mi impute chained` to impute various columns using MICE. Note that we also rename the existing preterm variable and create a (passive) preterm variable from the imputed gest_age columns

In [ ]:
local imputations = 20
qui {
    * make sure not already
    capture mi unset
    * set data for MI in wide format
    mi set wide
    * register various columns for imputation
    mi register imputed weight sex gest_age bmi mother_height
    * set seed for reproducibility
    set seed 123
    * impute all variables using MICE
    mi impute chained (regress) weight gest_age bmi mother_height (logit) sex = mother_age, add(`imputations') replace
    * generate preterm from the impute gestage
    rename preterm preterm_original
    mi passive: generate preterm = gest_age<37
}

## Data after imputation
Above, we have prepared our dataset for multiple imputation using the `mi` suite of commands.  The first command unsets the data in case it previously has been set. The second command sets our dataset to be ready for `mi` utilities, and indicates that any new columns (i.e. imputed data) should be appended as a new column (i.e. in wide format). We then register the variables that we would like to impute. In this case, we are imputing a combination of outcomes (`weight`, `gestage`) and non-outcomes (`sex`, `bmi`, `mother_height`) and for demonstration purposes, we are also adding a fixed indepedent var (i.e. a non-imputed variable): `mother_age`. For reproducibility, we set the seed (`set seed 123`), and then we run the MICE command, indicating that we want to perform linear regression when the equations includes continous variable, and logistic regression for the binary variables (in this case, `sex`)

After imputation, we can see that we have created a new column for each imputation and each imputed variable. For example for `weight` there are  columns `_1_weight, _2_weight, etc` for each imputation, and each of these columns hold values from *both* the original `weight` value and the imputed values

In [ ]:
ds

This is what those columns look like when `weight` is missing

In [ ]:
%head 2 weight _*_w* if weight==.

And of course, when `weight` is not missing, these values are constants.

In [ ]:
%head 2 weight _*_w* if weight!=.

## Analysis using multiply imputed datasets

The `mi` suite of commands allows for running estimation commands (like `regress`, `logit`, `poisson`, `xtgee`, etc). Therefore, if this were an individually randomized trial, we could estimate the treatment (i.e. ENP) effect on weight using imputed data, by simply doing something like:
```stata
mi estimate: regress weight facility_trt
```

Or, we could account for the clustered-randomization but still do an individual-level analysis using `xtgee` or `xtreg`, with or without adjustment, like this:
```stata
mi estimate: xtreg weight facility_trt sex mother_age mother_height bmi, mle i(cluster)
*or 
mi estimate: xtgee weight facility_trt sex, i(cluster)
```

However, there is no built-in cluster-level analysis command that can be passed as an estimation command to the `mi estimate: <estimation_command>` api. 

Therefore, we have two options
1. We can manually do the estimation on each of the imputed datasets, and combine the results according to Rubin's rules, perhaps using a user-written ado file to make this combining easier, and more reproducible
2. We can wrap our cluster-level analysis approaches in an ado file that has been custom-designed to work with the `mi estimate` api

## Manual estimation
Below, I demonstrate manual estimation of the effect of `ENP` on `weight` via cluster-level summaries, using multiple imputed datasets. The basic approach is to do the cluster-level analysis separately on each of the N imputed datasets in a loop, each time saving the results (basically the parameter estimate of interest, the standard error, and the degrees of freedom) into a separate accumulating frame, switch to that frame, and run a user-written command that can pool the results.  More specifically, the code below:

1. creates a frame (can be temporary, or permanent) to save the results of the each analysis
2. initiates a loop using `foreach` to loop through each of the N imputations
3. preserves the dataset 
4. estimates the mean weight for each cluster, using the `_<i>_weight` variable (i.e. we are doing this by imputed dataset)
5. does a regression of weight on facility_trt (i.e. equivalent to a t-test)
6. posts those results to the accumulating temporary frame
7. restores the dataset to return to the next imputed dataset, to repeat


In [ ]:
capture frame copy main_data cluster_level, replace
frame change cluster_level

capture frame drop manual_imputation_results


frame create manual_imputation_results double(imp beta variance df)
	
* Now, loop through each of the imputations
quietly {
    foreach i of numlist 1/`imputations' {
        * we are going to collapse, so we preserve first
        preserve
        * we regress the mean cluster-level weight on the allocation; this is equivalent
        * to doing a t-test
        collapse (mean) weight = _`i'_weight, by(facility_trt cluster_label)
        regress weight facility_trt
        * we post the imputation number, the estimate of the regression, the variance, and the residual
        * degrees of freedom to the temporary frame
        frame post manual_imputation_results (`i') (`=e(b)[1,1]') (`=e(V)[1,1]') (`=e(df_r)')
        * restore back to the non-collapsed version for the next iteration of the analysis
        restore
    }
}

## Results from each imputation
If we change to this frame, we can see the results that have accumulated (i.e. row-by-row) for each of the `N` imputed datasets. For each imputation, we have an estimate of the difference in cluster-level mean weight across `facility_trt`, and an estimate of the variance, plus the degrees of freedom (which is constant, because each imputation has the same number of clusters and same estimation command.)

In [ ]:
frame change manual_imputation_results
list

## How do we pool these? 
We can pool these following Rubin's Rules. The following ado file (called `"mi_pooled.ado"`, and located in the `../ado_files` folder), takes the names of the estimate column, the variance column, and the degrees of freedom column, and returns as set of scalar (`return list`), various values such as the overall estimate and the overall standard error, according to those Rules. We can manually then created CI from those values

```stata
program define mi_pooled, rclass
	version 16.1
	syntax varlist(min=2 max=3 numeric)
	tokenize `varlist'
	
	* summarize the first variable, which is the list of betas
	qui sum `1'
	* the pooled estimate is just mean of the betas
	tempname pe
	scalar `pe' = r(mean)
	* the between imputation variance is the variance of these betas
	tempname bv
	scalar `bv' = r(Var)
	
	* summarize the second variable, which is the list of variances
	qui sum `2'
	* the within imputation variance is the mean of the variances
	tempname wv
	scalar `wv' = r(mean)
	
	* the total variance is the within imputation variance plus the 
	* between imputation variance  + the between
	* imputation variance divided by the number of observation)
	tempname tv
	scalar `tv' = `wv' + `bv' + `bv'/`=_N'
	
	* degrees of freedom:
	tempname r
	scalar `r' = (`bv' + `bv'/`=_N')/`wv'
	tempname _df
	scalar `_df' = (`=_N'-1)*(1+1/`r')^2
	
	* 
	if "`3'"!="" {
	    qui sum `3'
		tempname v_complete
		scalar `v_complete' = r(min)
		tempname gam
		scalar `gam'= (1 + 1/`=_N')*`bv'/`tv'
		tempname v_obs
		scalar `v_obs' = `v_complete'*(`v_complete'+1)*(1-`gam')/(`v_complete' + 3)
		scalar `_df' = (1/`_df' + 1/`v_obs')^-1
	}
	
	* return these scalars
	return scalar b=`pe'
	return scalar V=`tv'
	return scalar wv = `wv'
	return scalar bv = `bv'
	return scalar m = `=_N'
	return scalar deg_freedom = `_df'

end
```

In [ ]:
mi_pooled beta variance df
return list

## One can leverage these returned scalars to manually construct a confidence interval

In [ ]:
local est = r(b)
local se = r(V)^0.5

* hard coding an alpha level
local level = 95
local alpha = (100-`level')/100
local lower = `est' - `se'*invt(r(deg_freedom), 1-`alpha'/2)
local upper = `est' + `se'*invt(r(deg_freedom), 1-`alpha'/2)
di "Est " %3.2f `est' " with `level'% CI: (" %3.2f `lower' " , " %3.2f `upper' ")"

## Better Option: writing our own `mi` compatible estimation command
The second option is to try to write an estimation command that is has `properties(mi)` making it compatible with `mi` and thus allowing us to call it using `mi estimate: <our_user-written_command>`. Below is an example of such a command

In [ ]:
capture program drop cluster_level_continuous

program define cluster_level_continuous, eclass properties(mi)
    syntax varlist(min=3 max=3 numeric)
    marksample touse
    tokenize `varlist'
    preserve 
        collapse (mean) `1', by(`2' `3')
        regress `1' `2'
        matrix b = e(b)
        matrix V = e(V)
        local n = _N
        local df_m = `n' - 2
        local df_r = `n' - 2
       
    restore
    ereturn post b V, esample(`touse')
    ereturn local cmd ="ttest"
    ereturn scalar N = `n'
    ereturn local title = "Continous Outcome, No Adjustment"
    ereturn scalar df_m = `df_m'
    ereturn scalar df_r = `df_r'
end
    
 

Once we have our command from above,  we can actually use it along with `mi estimate:`, which handles all of the above. That is, it figures out that `weight` is an imputed variable, and therefore run the `cluster_level_continuous` command once for each of the imputed datasets (like our loop), combines all the estimates together using Rubin's Rules (like our `mi_poooled` command), and uses the combined estimate, the between and within imputation variance, and the degrees of freedom to get a confidence interval (like our leveraging of the scalars returned by `mi_pooled`. Therefore, writing our own mi-compatible command is a better approach

<p style='color:red'>Notice that the results show below are equivalent to those obtained through our manual approach</p>

In [ ]:
capture frame copy main_data cluster_level, replace
frame change cluster_level
mi estimate: cluster_level_continuous weight facility_trt cluster_label

# Generalize the above command
Notice that above command only allows the outcome, the treatment variable, and the cluster variable to be passed (therefore, we can't use it for adjusment) and it only works for continous variables. Better if we could write the command in a much more general way so that it can handle a wide variety of questions. 

Here is one such implementation, and is stored in `ado_files/`
``` stata
capture program drop cluster_glm

program define cluster_glm, eclass properties(mi)
    syntax varlist(min=1 numeric), cluster(varname) trt(varname) [family(string),Level(real 95)]
    marksample touse
    tokenize `varlist'
    quietly {
        preserve
        if "`family'" == "binomial" {
            glm `varlist', family(`family') link(log)
            predict prob if e(sample)==1, xb
            replace prob = exp(prob)
            collapse (sum) expected = prob observed = `1', by(`trt' `cluster')
            gen cluster_y = observed/expected          
            qui ttest cluster_y, by(`trt') reverse
            noi t_test_binary_ci, level(`level')
            matrix b = (log(r(mu_1)) - log(r(mu_2))), log(r(mu_2))
            matrix colnames b = `trt' _cons
            matrix rownames b = y1
            scalar V = r(sd_1)^2/(r(N_1)*r(mu_1)^2) + r(sd_2)^2/(r(N_2)*r(mu_2)^2)
            scalar consV = r(sd_2)^2/(r(N_2)*r(mu_2)^2)
            matrix V = V, consV \ consV, -1*consV
            matrix rownames V = `trt' _cons
            matrix colnames V = `trt' _cons
            if "`2'" == "" local title = "Binary Outcome, No Adjustment"
            else local title = "Binary Outcome, Adjusted for Covariate(s)"
        } 
        else {
            glm `varlist', family(`family')
            predict resid if e(sample), working
            collapse (mean) cluster_y = resid, by(`trt' `cluster')
            noi regress cluster_y `trt', level(`level')
            matrix b = e(b)
            matrix V = e(V)
            if "`2'" == "" local title = "Continuous Outcome, No Adjustment"
            else local title = "Continuous Outcome, Adjusted for Covariate(s)"
        }
        
        local n = _N
        local df_m = 1
        local df_r = `n' - 2
        restore
    }

    ereturn post b V, esample(`touse')
    ereturn local cmd ="glm"
    ereturn scalar N = `n'
    ereturn local title = "`title'"
    ereturn scalar df_m = `df_m'
    ereturn scalar df_r = `df_r'

end
```

### Example, using generalized function for binary outcome, without adjustment
Note: for binary outcome, use `eform` sub-option in `mi estimate`. You can also set the confidence interval directly

In [ ]:
mi estimate,eform level(95): cluster_glm preterm, cluster(cluster_label) trt(facility_trt) family("binomial")

### Example, using generalized function for continuous outcome, without adjustment


In [ ]:
mi estimate: cluster_glm weight, cluster(cluster_label) trt(facility_trt)

### Example, using generalized function for continuous outcome, with adjustment

In [ ]:
mi estimate: cluster_glm weight bmi mother_age gest_age, cluster(cluster_label) trt(facility_trt)

<h2 style='color:red'>Note that we can even use this command in non-imputation situations</h2>
Compare the following calls to the main unadjusted results in the non-imputation notebook

In [ ]:
cluster_glm preterm_original, cluster(cluster_label) trt(facility_trt) family("binomial")

In [ ]:
cluster_glm weight, cluster(cluster_label) trt(facility_trt)